<a href="https://colab.research.google.com/github/samgregson/GHPT-colab-experiments/blob/main/Grasshopper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [2]:
if IN_COLAB:
    %pip install openai
    %pip install requests

if IN_COLAB:
    import os
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "GHPT_Instructor"

### load files into Colab and install package

NOTE: you may need to refresh your Colab files directory to see changes

In [3]:
if IN_COLAB:
    # remove the existing directory
    import shutil
    shutil.rmtree('/content/GHPT-experiments/', ignore_errors=True)
    %git clone "https://github.com/samgregson/GHPT-experiments"
    %pip install -e /content/GHPT-experiments/
    # add the modules to the search path
    import site
    site.main()

Load the correct SSL certificates

In [4]:
if not IN_COLAB:
    from dotenv import load_dotenv
    import os
    import ssl
    load_dotenv()
    context = ssl.create_default_context(cafile=os.environ.get("REQUESTS_CA_BUNDLE"))
    print(os.environ.get("REQUESTS_CA_BUNDLE"))

C:\Users\GRE111160\AppData\Roaming\ssl\ca-certificates.cer


test SSL certificate, this should return `<Response [200]>` if not try restarting the Jupyter kernel

In [16]:
import requests
requests.get("https://api.smith.langchain.com/docs")

<Response [200]>

### OpenAI

In [8]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

client = OpenAI()

# Custom wrap for VSCode, needs to be the first wrap!
if not IN_COLAB:
    from patch_openai.patch_openai import patch_openai
    client = patch_openai(client)

# Wrap the OpenAI client with LangSmith
client = wrap_openai(client)

# GHPT

### Import prompts

In [9]:
from prompts.generate_script import system_prompt, prompt_template

# replace all { with {{ and } with }} to escape the curly braces
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

# Instructor

In [10]:
if IN_COLAB:
    !pip install -U instructor
    !pip install anthropic

In [11]:
import instructor

client_instructor = instructor.patch(client, mode=instructor.Mode.TOOLS)

In [12]:
from langsmith import traceable
from models.models import GrasshopperScriptModel

# create the openai api call function
@traceable
def call_openai_instructor(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    completion = client_instructor.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_model=GrasshopperScriptModel,
        max_retries = 2
    )

    return completion

In [13]:
model = "gpt-3.5-turbo-1106"
# model = "gpt-4-1106-preview"
response = call_openai_instructor(prompt='Create a cone with a base radius of 5 and a height of 10', system_prompt=system_prompt)

In [14]:
print(response.json())

{"ChainOfThought":"Create a Point at the tip of the cone, Create a Circle at the base of the cone, Create a Cone using the Point and Circle as inputs","Additions":[{"Name":"Point","Id":1},{"Name":"Circle","Id":2},{"Name":"Cone","Id":3},{"Name":"Number Slider","Id":4},{"Name":"Number Slider","Id":5}],"Connections":[{"From":{"Id":1,"ParameterName":"Point"},"To":{"Id":3,"ParameterName":"Apex"}},{"From":{"Id":2,"ParameterName":"Circle"},"To":{"Id":3,"ParameterName":"Base"}},{"From":{"Id":4,"ParameterName":"Slider"},"To":{"Id":2,"ParameterName":"Radius"}},{"From":{"Id":5,"ParameterName":"Slider"},"To":{"Id":3,"ParameterName":"Height"}}],"Advice":"Adjust the Number Sliders to change the base radius and height of the cone."}
